In [119]:
from numpy import *
from pylab import *
from math import erf  # error function: erf(x)
import pandas as pd

In [120]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


D:\Program Files (x86)\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random', 'power', 'linalg', 'fft']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [121]:
# constant cell

r = 0.12  # risk-free interest
sigma = 0.09  # voltility
k = 50  # strike price

N = 300
s_start = 0
s_end = 150
s = linspace(s_start, s_end, N + 1)
ds = abs(s_end - s_start)/N  # ds, uniform grid

M = 6000
t_start = 0
t_end = 1  # expire time T
time = 0
t = linspace(t_start, t_end, M + 1)
dt = abs(t_end - t_start)/M  # dt, uniform grid

In [122]:
# list cell

# for all
V = [0 for i in range(N + 1)]  # exact solution V(s, T-t)
W = [0 for i in range(N + 1)]  # numerical solution W(s, T-t)
time_error = [0 for i in range(M + 1)]


# for midpoint formula
s_first_partial = [0 for i in range(N + 1)]  # first partial of W
s_second_partial = [0 for i in range(N + 1)]  # second partial of W

# for spline
b = [0 for i in range(N + 1)]  # coefficient vector b
c = [0 for i in range(N + 1)]  # coefficient vector c
row_vector = [0 for i in range(N + 1)]  # for vector equation A
row_vector[0] = ds; row_vector[1] = 4*ds; row_vector[2] = ds;  # consider N > 2
v_vector = [0 for i in range(N + 1)]  # for vector equation z
spl_matrix = []  # matrix A

In [123]:
# function cell

# for all

def normal_dist(x): # standard normal distribution
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

def CDF(t, idx):  # CDF of standard normal distribution
    arr = [0 for i in range(N + 1)]
    for i in range(1, N+1):
        if idx == 1:
            arr[i] = normal_dist((log(s[i]/k) + (r + sigma**2 / 2)*t)/(sigma*sqrt(t)))
        else:
            arr[i] = normal_dist((log(s[i]/k) + (r - sigma**2 / 2)*t)/(sigma*sqrt(t)))    
    return arr

def exact_func(t):  # exact(analytic) function for fixed t
    return s*float_(CDF(t,1)) - k*exp(-r*t)*float_(CDF(t,2))

def abs_error(V,W):  # absolute error
    return abs(V-W)

# for midpoint formula
def aprx_func(W):  # approximation function of midpoint formula
    for i in range(1,N):
        s_first_partial[i] = (W[i+1] - W[i-1])/(2*ds) # midpoint formula for first order
        s_second_partial[i] = (W[i+1] - 2*W[i] + W[i-1])/ds**2 # midpoint formula for second order
    
    return 1/2 * sigma**2 * (s*s)*s_second_partial + r*s*s_first_partial - r*float_(W)

# for spline
def rotate(arr, n): # rotate the given arr to right n times 
    n %= len(arr)
    if not n:
        return arr
    
    left = arr[:-n]
    right = arr[-n:]
    
    return right + left

In [124]:
# pandas cell

col = ['remaining time','stock price', 'exact', 'numerical', 'error']
row = ['']
data = [[0, 0, 0, 0, 0]]

In [125]:
# set the matrix A

row_end = [0 for i in range(N + 1)]
row_end[0] = 1
spl_matrix.append(row_end)
spl_matrix.append(row_vector)

for i in range(2,N):
    row_vector = rotate(row_vector,1)
    spl_matrix.append(row_vector)
    
row_end = [0 for i in range(N + 1)]
row_end[N] = 1
spl_matrix.append(row_end)

In [126]:
# set initial funciton
time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial graph
plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
title('W(x) at remaining time t')
legend()
axis((35, 65, 0, 21))
xlabel('stock price')
ylabel('call option price')

Text(0, 0.5, 'call option price')

In [44]:
# exact function

#initial function
time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        V[i] = s_start + i*ds - k
    else:
        V[i] = 0
        
# plot the graph        
plot(s,V, linewidth = 2.5, label = 'exact at t='+str(round(time, 2)))
title('V(x) at remaining time t')
legend()
axis((35, 65, 0, 21))
xlabel('stock price')
ylabel('call option price')


while time < t_end:
    time = time + dt
    
    V = exact_func(time)
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,V, linewidth = 2.5, label = 'exact at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        plot(s,V, linewidth = 2.5, label = 'exact at t='+str(round(time, 2)))
        legend()
                
    if isclose(time, 0.75):
        plot(s,V, linewidth = 2.5, label = 'exact at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,V, linewidth = 2.5, label = 'exact at t='+str(round(time, 2)))
        legend()

In [63]:
# Euler method for midpoint formula: numerical

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    W = W + dt*aprx_func(W)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        title('W(x) at remaining time t: midpoint Euler')
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()

In [ ]:
# Euler method for midpoint formula: error

time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
title('error of midpoint Euler')
legend()
axis((90, 150, 0, 0.0015))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    W = W + dt*aprx_func(W)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1.0):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()

In [ ]:
# RK2 for midpoint formula: numerical

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt/2 * RK1)
    
    W = W + dt*RK2
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()

In [ ]:
# RK2 for midpoint formula: error

time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
title('error of midpoint RK2')
legend()
axis((90, 150, 0, 0.0015))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt/2 * RK1)
    
    W = W + dt*RK2
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()

In [ ]:
# RK3 for midpoint formula: numerical

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt * RK1)
    RK3 = aprx_func(W + dt/2 * (RK1 + RK2)/2)
    
    W = W + dt/6 * (RK1 + RK2 + 4*RK3)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]

    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()

In [ ]:
# RK3 for midpoint formula: error

time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
title('error of midpoint RK3')
legend()
axis((90, 150, 0, 0.05))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt * RK1)
    RK3 = aprx_func(W + dt/2 * (RK1 + RK2)/2)
    
    W = W + dt/6 * (RK1 + RK2 + 4*RK3)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()

In [ ]:
# RK4 for midpoint formula: numerical

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt/2 * RK1)
    RK3 = aprx_func(W + dt/2 * RK2)
    RK4 = aprx_func(W + dt * RK3)
    
    W = W + dt/6 * (RK1 + 2*RK2 + 2*RK3 + RK4)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()

In [ ]:
# RK4 for midpoint formula: error

time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
title('error of midpoint RK4')
legend()
axis((90, 150, 0, 0.05))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical solution
    RK1 = aprx_func(W)
    RK2 = aprx_func(W + dt/2 * RK1)
    RK3 = aprx_func(W + dt/2 * RK2)
    RK4 = aprx_func(W + dt * RK3)
    
    W = W + dt/6 * (RK1 + 2*RK2 + 2*RK3 + RK4)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()

In [127]:
# Euler method for spline

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    W = W + dt*(r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W))
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()

In [10]:
# Euler method for spline: error

time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
title('error of spline Euler')
legend()
axis((90, 150, 0, 0.0008))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    W = W + dt*(r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W))
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
# RK2 for spline

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    RK1 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W)
    RK2 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt/2 * RK1)
    
    W = W + dt*RK2
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()

In [ ]:
# RK2 for spline: error

time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
title('error of spline RK2')
legend()
axis((90, 150, 0, 0.05))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    RK1 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W)
    RK2 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt/2 * RK1)
    
    W = W + dt*RK2
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()

In [ ]:
# RK3 for spline

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    RK1 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W)
    RK2 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt * RK1)
    RK3 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt/2 * (RK1 + RK2)/2)
    
    W = W + dt/6 * (RK1 + RK2 + 4*RK3)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()

In [ ]:
# RK3 for spline: error

time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
title('error of spline RK3')
legend()
axis((90, 150, 0, 0.05))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    RK1 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W)
    RK2 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt * RK1)
    RK3 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt/2 * (RK1 + RK2)/2)
    
    W = W + dt/6 * (RK1 + RK2 + 4*RK3)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()

In [ ]:
# RK4 for spline

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    RK1 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W)
    RK2 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt/2 * RK1)
    RK3 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt/2 * RK2)
    RK4 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt * RK3)
    
    W = W + dt/6 * (RK1 + 2*RK2 + 2*RK3 + RK4)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the graph
    if isclose(time, 0.25):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        plot(s,W, linewidth = 2.5, label = 'numerical at t='+str(round(time, 2)))
        legend()

In [ ]:
# RK4 for spline: error

time = 0
clf()
for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0
        
# plot the initial error
error = [0 for i in range(N+1)]
plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
title('error of spline RK4')
legend()
axis((90, 150, 0, 0.05))
xlabel('stock price')
ylabel('error')

while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    RK1 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W)
    RK2 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt/2 * RK1)
    RK3 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt/2 * RK2)
    RK4 = r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W + dt * RK3)
    
    W = W + dt/6 * (RK1 + 2*RK2 + 2*RK3 + RK4)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    # plot the error
    if isclose(time, 0.25):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.5):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 0.75):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()
        
    if isclose(time, 1):
        V = exact_func(time)
        error = abs_error(V,W)
        plot(s,error, linewidth = 2.5, label = 't='+str(round(time, 2)))
        legend()

In [133]:
# set initial funciton
time = 0

for i in range(N+1):
    if (s_start + i*ds) >= k:
        W[i] = s_start + i*ds - k
    else:
        W[i] = 0

In [131]:
# midpoint global error

e = 1
clf()
while time < t_end:
    time = time + dt
    
    # find the numerical solution
    W = W + dt*aprx_func(W)
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)
    
    time_error[M-e] = sum(abs(V-W))
    e = e+1
    
time_error[M] = 0
plot(t,time_error,linestyle='--',linewidth = 1.8, label='Euler midpoint')
legend()

In [134]:
# spline global error

e = 1
while time < t_end:
    time = time + dt
    
    # find the numerical constant vector
    for i in range(1,N):
        v_vector[i] = 3/ds*(W[i+1] - 2*W[i] + W[i-1])
    v_vector[0] = 0; v_vector[N] = 0
    
    c = linalg.solve(spl_matrix, v_vector)  # solve the vector equation
    
    c[0] = 0; c[N] = 0
    
    for i in range(N):
        b[i] = (W[i+1]-W[i])/ds - ds/3*(2*c[i]+c[i+1])
    b[0] = 0; b[N] = 1
    
    # find the numerical solution
    W = W + dt*(r*s*b + 1/2*sigma**2*s**2*2*c - r*float_(W))
    W[0] = 0; W[N] = (W[N-1]-W[N-2]) + W[N-1]
    
    V = exact_func(time)
    
    time_error[M-e] = sum(abs(V-W))
    e = e+1
    
time_error[M] = 0
plot(t,time_error,linestyle='--',linewidth = 1.8, label='Euler spline')
legend()